In [8]:
import pandas as pd
import requests
from io import StringIO
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.metrics import accuracy_score

url = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv'
response = requests.get(url)
data = StringIO(response.text)
spotify_songs = pd.read_csv(data)
spotify_songs = spotify_songs.dropna()
spotify_songs['track_album_release_date'] = pd.to_datetime(spotify_songs['track_album_release_date'], errors='coerce')
spotify_songs['release_year'] = spotify_songs['track_album_release_date'].dt.year.astype('Int64')
spotify_songs = spotify_songs.drop(columns=['track_album_release_date'])

# Drop undesired columns
undesired_columns = ['playlist_id', 'playlist_name', 'track_id', 'track_name', 'track_album_name', 'track_album_id', 'track_artist', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']
spotify_songs = spotify_songs.drop(columns=undesired_columns)

# Add a 'decade' column
spotify_songs['decade'] = (spotify_songs['release_year'] // 10) * 10

# Filter rows with years between 1960 and 2019
spotify_songs = spotify_songs[(spotify_songs['release_year'] >= 1960) & (spotify_songs['release_year'] <= 2019)]
spotify_songs = spotify_songs.drop(columns=['release_year'])

# Filter songs with popularity greater than 50
popular_songs = spotify_songs[spotify_songs['track_popularity'] > 50]

# Make 'decade' the target variable
X = popular_songs.drop(['decade', 'track_popularity'], axis=1)
y = popular_songs['decade']

# Identify all categorical columns
categorical_columns = X.select_dtypes(include=['object']).columns.tolist()

# One-hot encode all categorical columns
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_encoded = pd.DataFrame(encoder.fit_transform(X[categorical_columns]))
X_encoded.columns = encoder.get_feature_names_out(categorical_columns)

# Resetting index to ensure consistency
X_encoded.reset_index(drop=True, inplace=True)
X.reset_index(drop=True, inplace=True)

# Drop the original categorical columns and concatenate the encoded ones
X = pd.concat([X.drop(categorical_columns, axis=1), X_encoded], axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a decision tree classifier
clf = DecisionTreeClassifier()

# Train the classifier
clf.fit(X_train, y_train)

# Display the decision tree rules
tree_rules = export_text(clf, feature_names=X.columns.tolist())
print(tree_rules)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

|--- playlist_genre_rock <= 0.50
|   |--- playlist_subgenre_new jack swing <= 0.50
|   |   |--- playlist_subgenre_southern hip hop <= 0.50
|   |   |   |--- playlist_subgenre_neo soul <= 0.50
|   |   |   |   |--- playlist_subgenre_gangster rap <= 0.50
|   |   |   |   |   |--- playlist_genre_edm <= 0.50
|   |   |   |   |   |   |--- playlist_subgenre_trap <= 0.50
|   |   |   |   |   |   |   |--- playlist_subgenre_indie poptimism <= 0.50
|   |   |   |   |   |   |   |   |--- playlist_subgenre_tropical <= 0.50
|   |   |   |   |   |   |   |   |   |--- playlist_subgenre_dance pop <= 0.50
|   |   |   |   |   |   |   |   |   |   |--- playlist_genre_rap <= 0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 7
|   |   |   |   |   |   |   |   |   |   |--- playlist_genre_rap >  0.50
|   |   |   |   |   |   |   |   |   |   |   |--- class: 2010.0
|   |   |   |   |   |   |   |   |   |--- playlist_subgenre_dance pop >  0.50
|   |   |   |   |   |   |   |   |   |   |--- class: 

c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
